In [1]:
from keras.layers import UpSampling2D
import matplotlib.pyplot as plt
import time
import pickle
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, Add
from tensorflow.keras.layers import Conv2DTranspose, DepthwiseConv2D, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

# Commonly used modules
import numpy as np
import os
import sys
import cv2

# Images, plots, display, and visualization
import matplotlib.pyplot as plt

print(tf.__version__)

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# instances = []
# PreOptimizationImages = []

# # Load the images from the directory
# count = 0
# for filename in os.listdir(unzip_path):
#     count += 1
#     filepath = os.path.join(unzip_path, filename)
#     print("filepath: ", filepath)
#     image = Image.open(filepath).convert('L')  # Open image and convert to grayscale
#     PreOptimizationImages.append(np.array(image))

# print("Saved images:", len(PreOptimizationImages))
# print(type(PreOptimizationImages[0]))
# plt.imshow(PreOptimizationImages[0], cmap='gray')  # Display grayscale image


Mounted at /content/drive
2.12.0


In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np

# Set the paths to the image folders
pre_optimization_folder = '/content/drive/MyDrive/CAL/Data/June 7 2023/preProjectionSeq'
post_optimization_folder = '/content/drive/MyDrive/CAL/Data/June 7 2023/ImageSeq'

# Function to load grayscale images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.png'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
    return images

# Load pre-optimization grayscale images
pre_optimization_images = load_images_from_folder(pre_optimization_folder)

# Load post-optimization grayscale images
post_optimization_images = load_images_from_folder(post_optimization_folder)

# Convert image lists to numpy arrays
pre_optimization_images = np.array(pre_optimization_images)
post_optimization_images = np.array(post_optimization_images)

# Normalize the image data
pre_optimization_images = pre_optimization_images / 255.0
post_optimization_images = post_optimization_images / 255.0

# Print the shape of the loaded image arrays
print("Pre-optimization images shape:", pre_optimization_images.shape)
print("Post-optimization images shape:", post_optimization_images.shape)


Pre-optimization images shape: (9360, 128, 128)
Post-optimization images shape: (9396, 128, 128)


In [3]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#    raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [4]:
# #Build the model
# dropout = 0.2
# input = tf.keras.Input(shape=x_train[0].shape)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate


# model based off of https://www.youtube.com/watch?v=GAYJ81M58y8&t=910s
def unet(input_shape):
    # Contracting path
    inputs = Input(input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Expanding path
    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv9)

    # Output layer
    output = Conv2D(1, 1, activation='softmax')(conv9)

    # Create the model
    model = Model(inputs=inputs, outputs=output)

    return model

# print(PreOptimizationImages[0].shape)
# for layer in range(PreOptimizationImages[0].shape[2]):
#     layer_array = PreOptimizationImages[0][:,:,layer]
#     non_zero_count = np.count_nonzero(layer_array)
#     print("Non-zero count for layer", layer+1, ":", non_zero_count)



# input_shape = (256, 256, 1)  # Shape of the input images
input_shape = pre_optimization_images[0].shape + (1,)  # Add the channel dimension
print("Input shape:", input_shape)
model = unet(input_shape)
model.summary()


Input shape: (128, 128, 1)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  36928       ['conv2d[0][0]']                 
                                )                                  

In [5]:
# NAME = "depth-cnn-output-dense-" + str(time.time())
# #tensorboard = TensorBoard(log_dir="C:/Users/sotao/Desktop/Data/logs/" + NAME)

# #model.load_weights("C:/Users/sotao/Desktop/Data/Checkpoints/depth-cnn-output-dense-1627746842.3250527") #FIND THE NAME
# # model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
# #     filepath= "C:/Users/sotao/Desktop/Data/Checkpoints/" + NAME,
# #     monitor='val_loss',
# #     save_best_only = False,
# #     verbose = 1,
# #     save_weights_only = True)

# model.compile(optimizer=tf.keras.optimizers.Adam(0.0005),
#               loss='mean_squared_error',
#               metrics=[tf.keras.metrics.RootMeanSquaredError()])


# # Train model below
# # history = model.fit(x_train, y_train, epochs=1, validation_split = 0.2, shuffle = True, callbacks=[tensorboard])


from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    pre_optimization_images, post_optimization_images, test_size=0.2, random_state=42
)

# Prepare the data for training
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
x_train = x_train / 255.0  # Normalize pixel values
x_test = x_test / 255.0

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))


ValueError: ignored